In [15]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns
from datetime import datetime,timedelta
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from xgboost import XGBClassifier
import datetime
import holidays
from sklearn.preprocessing import LabelEncoder
import prophet
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor, plot_importance
from pre_processing_sy import *
import random
import os
import torch
from torch.autograd import Variable
import xgboost
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from random import randint
from catboost import CatBoostRegressor
# from sktime.forecasting.model_selection import temporal_train_test_split
# from sktime.utils.plotting import plot_series

In [16]:
def seed_everything(seed: int = 2024):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
seed_everything(2024)

### 데이터 불러오기

In [17]:
# 데이터 불러오기
train = pd.read_csv("dataset/train.csv")
test = pd.read_csv("dataset/test.csv")
trade = pd.read_csv("dataset/international_trade.csv")
submission = pd.read_csv("dataset/sample_submission.csv")


train_pre, test_pre = pre_all(train, test)

# # 공휴일이지만 안쉬는 날 제외하기
no_holi = list(train_pre[(train_pre["item"] =="TG") &(train_pre["holiday"]==1) & (train_pre["price"]!=0)].groupby("timestamp").count().reset_index()["timestamp"])
noholi_idx = train_pre[train_pre["timestamp"].isin(no_holi)]["holiday"].index
for idx in noholi_idx:
    train_pre.loc[idx, "holiday"] = 0
    test_pre.loc[idx, "holiday"] = 0

print(f"train의 컬럼 : {train_pre.columns}")
print(f"test의 컬럼 : {test_pre.columns}")

train_tg = train_pre[train_pre["item"] =="TG"]
test_tg = test_pre[test_pre["item"] == "TG"]

Xy = pd.get_dummies(train_tg.sort_values(by = ["timestamp", "corporation","location"]).reset_index(drop=True).drop(columns = ["item","supply"]), columns = [ "corporation","location"])
answer = pd.get_dummies(test_tg.drop(columns = ["timestamp","supply","price","item",]), columns = [ "corporation","location"])
print(Xy.columns)

# 종속변수 루트값
Xy["price"] = np.sqrt(Xy["price"])


전처리 전 train 크기 : (59397, 7)
전처리 전 test 크기 : (1092, 5)
=================전처리 중=================
전처리 후 train 크기 : (59397, 15)
전처리 후 test 크기 : (1092, 15)
train의 컬럼 : Index(['ID', 'timestamp', 'item', 'corporation', 'location', 'supply', 'price',
       'year', 'month', 'day', 'week_day', 'year_month', 'week', 'week_num',
       'holiday'],
      dtype='object')
test의 컬럼 : Index(['ID', 'timestamp', 'item', 'corporation', 'location', 'supply', 'price',
       'year', 'month', 'day', 'week_day', 'year_month', 'week', 'week_num',
       'holiday'],
      dtype='object')
Index(['ID', 'timestamp', 'price', 'year', 'month', 'day', 'week_day',
       'year_month', 'week', 'week_num', 'holiday', 'corporation_A',
       'corporation_B', 'corporation_C', 'corporation_D', 'corporation_E',
       'location_J', 'location_S'],
      dtype='object')


In [18]:
print(f"Xy컬럼 : {Xy.columns}")
print(f"test컬럼 : {answer.columns}")

Xy컬럼 : Index(['ID', 'timestamp', 'price', 'year', 'month', 'day', 'week_day',
       'year_month', 'week', 'week_num', 'holiday', 'corporation_A',
       'corporation_B', 'corporation_C', 'corporation_D', 'corporation_E',
       'location_J', 'location_S'],
      dtype='object')
test컬럼 : Index(['ID', 'year', 'month', 'day', 'week_day', 'year_month', 'week',
       'week_num', 'holiday', 'corporation_A', 'corporation_B',
       'corporation_C', 'corporation_D', 'corporation_E', 'location_J',
       'location_S'],
      dtype='object')


In [ ]:
# 그리드 서치 결과
tg_cat_grid = pd.read_csv("[1117]tg_cat_gridsearch_timecorlocsort.csv")

In [19]:
# 예측 및 
n_estimators =1000
lrs = 0.05
max_depths = 10
l2_leaf_reg = 3

cat = CatBoostRegressor(random_state = 2024, 
                                n_estimators = n_estimators, 
                                learning_rate = lrs, 
                                depth = max_depths, 
                                l2_leaf_reg = l2_leaf_reg,
                                metric_period = 1000)

cat.fit(Xy.drop(columns = ["timestamp", "ID","price"]), Xy["price"])

pred = cat.predict(answer.drop(columns = ["ID"]))
for idx in range(len(pred)):
    if pred[idx]<0:
        pred[idx]= 0
answer["answer"] = np.power(pred,2)

answer[["ID","answer"]]

0:	learn: 30.2823698	total: 11.8ms	remaining: 11.8s
999:	learn: 11.0655757	total: 9.04s	remaining: 0us


,ID,answer
0,TG_A_J_20230304,2742.172780
1,TG_E_S_20230304,3354.003496
3,TG_E_J_20230304,760.581106
5,TG_D_S_20230304,3769.615969
8,TG_D_J_20230304,53.451592
...,...,...
1074,TG_D_S_20230331,4727.895634
1077,TG_A_S_20230331,5403.967583
1079,TG_E_S_20230331,4715.201735
1087,TG_A_J_20230331,6813.935154


In [20]:
answer[["ID","answer"]].sort_values(by=["ID"], ascending = False)

,ID,answer
1079,TG_E_S_20230331,4715.201735
1044,TG_E_S_20230330,4636.905040
1006,TG_E_S_20230329,4319.457891
957,TG_E_S_20230328,4406.550254
928,TG_E_S_20230327,4533.301873
...,...,...
180,TG_A_J_20230308,3528.164347
122,TG_A_J_20230307,3766.824294
102,TG_A_J_20230306,3030.469778
54,TG_A_J_20230305,28.367627


In [7]:
# noindex 와 합치기
dc_677 = pd.read_csv("submit/answer1112_noindex.csv")
dc_677["item"] = dc_677["ID"].map(lambda x:x.split("_")[0])
dc = dc_677[dc_677["item"] != "TG"]
answers = pd.concat([dc, answer[["ID","answer"]]]).drop(columns = ["item"])

In [12]:
answers.to_csv("submit/1117tg_grid_sort.csv")